## Att leka med kluster av mat

Det finns över tvåtusen livsmedel i Livsmedelsdatabasen, och de har värden för över 50 näringsvärden.

Att sortera dem i Excel funkar ett tag, men det blir svårt att få en samlad bedömning eller gruppering av stora mängder livsmedel.

Det finns också en metadataklassificering som Livsmedelsverket kallar "huvudgrupp". Det är en klassificering som bara har en nivå. Inget över, inget under. En flat nivå.

In [422]:
import sqlite3
import numpy as np
from sklearn.cluster import KMeans
import plotly
import plotly.graph_objs as go
import plotly.plotly as py

In [423]:
debug = True

Börja med att läsa in allt från databasen. Det är inte så mycket...

In [424]:
conn = sqlite3.connect('livs.db')
conn.row_factory = sqlite3.Row
curs = conn.cursor()

result = []
for row in curs.execute('select * from livs'):
    result.append(row)

conn.close()

dataset = np.array(result)
if debug:
    print (dataset[:5,:5])

[['Talg nöt' '1' 656.3 2746.0 0.0]
 ['Späck gris' '2' 763.0 3192.6 0.0]
 ['Ister gris' '3' 884.3 3700.0 0.0]
 ['Kokosfett' '4' 884.3 3700.0 0.0]
 ['Matfettsblandning havssaltat fett 80% berikad typ Bregott' '5' 711.5
  2977.0 0.5]]


Livsmedelsverkets gruppering har jag lagt i kolumn 60. Numpy-funktionen np.unique() returnerar både alla unika värden och (om man vill) hur många som är i varje kategori.

In [484]:
unique, counts = np.unique(dataset.T[60], return_counts=True) #https://stackoverflow.com/questions/28663856/how-to-count-the-occurrence-of-certain-item-in-an-ndarray-in-python
huvudgrupp_storlek = np.array(list(zip(unique, counts)))

Skriver man ut huvudgrupp_storlek blir det typ 118 kategorier i bokstavsordning. Här är de n första

In [486]:
n = 20
print (huvudgrupp_storlek[:n])

[['' '7']
 ['Algprodukter' '1']
 ['Baljväxter (bönor, linser och ärter)' '51']
 ['Blodmat' '2']
 ['Blodprodukter o rätter' '8']
 ['Buljong' '18']
 ['Bullar kakor tårtor mm' '86']
 ['Bär färska frysta' '24']
 ['Chips popcorn o dyl' '11']
 ['Choklad' '7']
 ['Cider alkoläsk drink' '3']
 ['Deg och gräddade skal och bottnar' '3']
 ['Dessertost' '8']
 ['Dryck' '1']
 ['Efterrätter' '31']
 ['Fisk färsk fryst kokt' '68']
 ['Fisk o skaldjursprodukter o rätter' '69']
 ['Fisk rökt' '13']
 ['Fisk stekt ej panerad' '7']
 ['Flingor - frukostflingor' '28']]


Jag förstår inte riktigt varför det är så svårt att sortera den arrayen. Det är ju en sak att allting är strängar, eftersom en numpy-array bara kan ha en enda datatyp, men det borde finnas ett enkelt sätt att sortera efter kolumn två som integer...

Som det blir nu behöver man skapa ett index genom att använda np.argsort, ta de 20 första med [-20:] och sen vända på ordningen med [::-1], som helt enkelt är en slice till, där -1 är step-argumentet. https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.indexing.html 

In [518]:
antal = 20
index = np.argsort(huvudgrupp_storlek.T[1].astype(int))[-antal:][::-1] #https://stackoverflow.com/questions/6910641/how-to-get-indices-of-n-maximum-values-in-a-numpy-array
print(huvudgrupp_storlek[index])

[['Kött färskt fryst tillagat' '127']
 ['Grönsaker' '94']
 ['Bullar kakor tårtor mm' '86']
 ['Köttprodukter kötträtter' '80']
 ['Sås dressing majonnäs' '79']
 ['Fisk o skaldjursprodukter o rätter' '69']
 ['Fisk färsk fryst kokt' '68']
 ['Korv' '60']
 ['Grönsaks- rotfrukts- baljväxträtter och produkter' '55']
 ['Baljväxter (bönor, linser och ärter)' '51']
 ['Soppa mat' '47']
 ['Pizza paj pirog färdig smörgås' '46']
 ['Frukt färsk fryst' '42']
 ['Mjukt bröd' '42']
 ['Hård matfettsblandning' '39']
 ['Fågel' '39']
 ['Potatisprodukter o rätter' '33']
 ['Inälvor och organ' '33']
 ['Potatis' '31']
 ['Efterrätter' '31']]


Medan de största grupperna är ganska rättframma är de minsta desto knepigare. Vad ska man göra med en klassificering som innehåller en huvudgrupp "Risrätter" med endast ett livsmedel, och det är paella och inte exempelvis risgrynsgröt?

In [519]:
index = np.argsort(huvudgrupp_storlek.T[1].astype(int))[:20] #https://stackoverflow.com/questions/6910641/how-to-get-indices-of-n-maximum-values-in-a-numpy-array
print(huvudgrupp_storlek[index])

[['Sötningsmedel' '1']
 ['Algprodukter' '1']
 ['Kryddor' '1']
 [ 'Övrigt animaliskt *kött*, grodlår, sniglar, säl - färskt, fryst, tillagat'
  '1']
 ['Sockerfritt godis' '1']
 ['Risrätter' '1']
 ['Dryck' '1']
 ['Tacoskal' '1']
 ['Riskakor' '2']
 ['Frukt o nötblandningar bars' '2']
 ['Frukt o bär' '2']
 ['Sportdrycker energidrycker' '2']
 ['Gelatin agar agar' '2']
 ['Övrigt' '2']
 ['Blodmat' '2']
 ['Tuggummi' '2']
 ['Kakaoprodukter' '2']
 ['Cider alkoläsk drink' '3']
 ['Osträtter' '3']
 ['Smör' '3']]


Det går ju även att få fram vilka livsmedel som finns i de där grupperna.

In [520]:
for row in huvudgrupp_storlek[index]:
    q = np.where(dataset.T[60]==row[0])
    print(row[0],"\n",dataset[q,0]) #Det går ju att skriva ut vilka värden som helst för varje livsmedel, som med dataset[q,:]

Sötningsmedel 
 [['Sorbitol m sackarin']]
Algprodukter 
 [['Kelp torkad']]
Kryddor 
 [['Kryddblandning taco']]
Övrigt animaliskt *kött*, grodlår, sniglar, säl - färskt, fryst, tillagat 
 [['Grodlår råa frysta']]
Sockerfritt godis 
 [['Karameller syrliga sockerfria']]
Risrätter 
 [['Paella']]
Dryck 
 [['Mandeldryck']]
Tacoskal 
 [['Tacoskal']]
Riskakor 
 [['Riskakor fullkorn solrosfrön majs fett 4%'
  'Riskakor fullkorn smaksatta fett 18%']]
Frukt o nötblandningar bars 
 [['Energibar choklad nötter Start'
  'Müslibar fullkorn berikad typ Special K Bar Red fruit']]
Frukt o bär 
 [['Havtorn' 'Aronia']]
Sportdrycker energidrycker 
 [['Sportdryck drickf' 'Energidryck typ Red Bull berikad']]
Gelatin agar agar 
 [['Agar torkad' 'Gelatinblad gelatinpulver']]
Övrigt 
 [['Samarinpulver' 'Samarin drickf']]
Blodmat 
 [['Gris blod rå' 'Nöt blod rå']]
Tuggummi 
 [['Tuggummi' 'Tuggummi sockerfritt']]
Kakaoprodukter 
 [['Kakaopulver fett 20-22%' "Kakaopulver m socker fett 2,5% typ O'boy"]]
Cider alkol

## Klustring av näringsvärden
Den här klassificeringen är alltså den som Livsmedelsverket själva har gjort, och det går att ändra parametrarna här ovan och få fram mycket mer om den.

Men vad händer om man baserar en gruppering bara på näringsvärden i stället? 

Det finns över 50 näringsvärden i databasen.

Av de över 50 näringsvärdena får man välja ut ett antal som man tror kan spela roll för att ge livsmedlen en viss "profil". 

I princip skulle man kunna ha alla näringsvärden med, och man kan testa sig fram. Men på något sätt tror jag att de näringsvärden där många livsmedel har noll-värden kommer att förvirra klustringen.
```
[2 'Energi_kcal' 'REAL' 0 None 0]
[4 'Kolhydrater_g' 'REAL' 0 None 0]
[5 'Fett_g' 'REAL' 0 None 0]
[6 'Protein_g' 'REAL' 0 None 0]
[7 'Fibrer_g' 'REAL' 0 None 0]
[8 'Vatten_g' 'REAL' 0 None 0]
[9 'Alkohol_g' 'REAL' 0 None 0]
[10 'Aska_g' 'REAL' 0 None 0]
[42 'Vitamin_C_mg' 'REAL' 0 None 0]
[50 'Jarn_mg' 'REAL' 0 None 0]
```

In [425]:
#Här är det urval som används nu
relevant_columns = [ 2,  4,  5,  6,  7,  8,  9, 10, 42, 50]

#Här är alla möjliga dimensioner. En del försvinner eftersom de 
#har värden som saknas för vissa livsmedel
#[ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57]

I numpy kan man använda en array som index, men det gäller att göra det till ett index för kolumnerna och inte för raderna... (Läs mer om array som index här: <http://infontology.typepad.com/infontokod/2017/11/bra-funktioner-i-numpy.html> Där står också om den här "slice"-notationen array[a:b,c:d])

In [426]:
columns = dataset[:,relevant_columns] #Tar man bort :, blir det alltså ett urval rader
columns

array([[656.3, 0.0, 71.0, ..., 0.3, 0.0, 0.3],
       [763.0, 0.0, 85.0, ..., 0.7, 0.0, 0.2],
       [884.3, 0.0, 100.0, ..., 0.0, 0.0, 0.1],
       ..., 
       [40.6, 7.6, 0.5, ..., 0.2, 0.0, 0.1],
       [372.6, 72.6, 3.55, ..., 0.0, 0.0, 11.35],
       [313.7, 57.5, 4.8, ..., 0.0, 0.0, 0.85]], dtype=object)

Det går förstås att välja en massa olika klustringsalgoritmer. Vi har valt k-means. Det finns också en hel del parametrar att välja. <http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html>

In [427]:
 def cluster(dataset):
    kmeans = KMeans(n_clusters=numClusters)
    kmeans.fit(dataset)

    centroids = kmeans.cluster_centers_
    clusters = kmeans.labels_
    
    #Utskriften kommer ju inte här, utan efter funktionsanropet
    if debug:
        print (centroids)
        print (clusters)

    return clusters, centroids

In [428]:
#Livsmedelsdatabasen innehåller över 100 huvudgrupper. Det hade varit bra att ha något 
#färre grupper på översta nivån. Högst 50. Eller kanske bara 10, för att komma ner i något som
#skulle kunna motsvara "frukt, grönsaker, fisk, kött, nötter" etc.
numClusters = 10

In [429]:
clusters, centroids = cluster(columns)

[[  3.65919474e+02   6.83748421e+01   4.52452632e+00   9.07021053e+00
    5.88478947e+00   1.01441053e+01   4.99600361e-16   2.01000000e+00
    4.96684211e+00   3.33884211e+00]
 [  1.25491607e+02   7.70496403e+00   4.66539568e+00   1.23515588e+01
    8.98201439e-01   7.23059233e+01   1.32374101e-01   1.90901679e+00
    3.15611511e+00   1.53479616e+00]
 [  2.64772619e+02   2.32418254e+01   1.39476984e+01   1.01683333e+01
    2.10753968e+00   4.66275794e+01   2.30158730e-01   3.58988095e+00
    2.75119048e+00   1.70107143e+00]
 [  8.78417647e+02   2.41176471e-02   9.92829412e+01   8.58823529e-02
    4.11764706e-03   5.37058824e-01   8.32667268e-17   4.82352941e-02
    5.29411765e-02   2.64705882e-02]
 [  6.67910204e+02   4.74183673e+00   6.98132653e+01   6.43877551e+00
    2.76714286e+00   1.43638776e+01   5.55111512e-17   1.84163265e+00
    3.65306122e-01   1.48755102e+00]
 [  3.13980263e+01   4.74026316e+00   4.30526316e-01   1.25207237e+00
    1.45891447e+00   9.00242434e+01   6.45065

In [431]:
clusters

array([4, 4, 3, ..., 5, 0, 0], dtype=int32)

In [437]:
unique, counts = np.unique(clusters, return_counts=True) #https://stackoverflow.com/questions/28663856/how-to-count-the-occurrence-of-certain-item-in-an-ndarray-in-python
cluster_distribution = np.array(list(zip(unique, counts)))
print(cluster_distribution)
print("Medelvärde:", np.mean(cluster_distribution.T[1]))
print("Median:", np.median(cluster_distribution.T[1]))
print("Standardavvikelse:",np.std(cluster_distribution.T[1]))

[[  0 190]
 [  1 417]
 [  2 252]
 [  3  17]
 [  4  49]
 [  5 304]
 [  6 304]
 [  7 112]
 [  8 349]
 [  9  94]]
Medelvärde: 208.8
Median: 221.0
Standardavvikelse: 129.700269853


In [438]:
ind=np.where(cluster_distribution.T[1]>40)
ind

(array([0, 1, 2, 4, 5, 6, 7, 8, 9]),)

    Hopp  

In [403]:
compliance=[]
for item in ind[0]:
    print("Kluster " + str(item))
    #print(dataset[np.where(clusters==item)][:,[0,60]])
    a=dataset[np.where(clusters==item)][:,[0,60]]
    #print(a[0][1])
    unique, counts = np.unique(a.T[1], return_counts=True) #https://stackoverflow.com/questions/28663856/how-to-count-the-occurrence-of-certain-item-in-an-ndarray-in-python
    huvudgrupp_distribution = np.array(list(zip(unique, counts)))
    order=np.argsort(huvudgrupp_distribution.T[1].astype(int))
    order = np.flipud(order)
    #print (order)
    huvudgrupp_distribution = huvudgrupp_distribution[order]
    totals = np.array([])
    for item in huvudgrupp_distribution:
        a=np.where(huvudgrupp_storlek.T[0] == item [0])
        #print(a[0], huvudgrupp_storlek[a[0],1], item[0])
        totals=np.append(totals,huvudgrupp_storlek[a[0],1])
    samlad_array = np.concatenate((huvudgrupp_distribution.T,[totals]),axis=0).T
    #print(np.shape(huvudgrupp_distribution), np.shape([totals]))
    #print(huvudgrupp_distribution)
    #print("totals" + str(totals.T))
    print()
    part = lambda x: np.divide(x[1].astype(int),x[2].astype(int))
    andel = part(samlad_array.T)
    samlad_array = np.vstack((samlad_array.T,andel.T))
    print (samlad_array.T)
    compliance.append(andel)
    

Kluster 0

[['Potatis' '11' '31' '0.3548387096774194']
 ['Grönsaks- rotfrukts- baljväxträtter och produkter' '6' '55'
  '0.10909090909090909']
 ['Frukt färsk fryst' '6' '42' '0.14285714285714285']
 ['Söta soppor kräm o efterrättssås' '3' '26' '0.11538461538461539']
 ['Frukt o bär konserverade' '3' '17' '0.17647058823529413']
 ['Bär färska frysta' '3' '24' '0.125']
 ['Baljväxter (bönor, linser och ärter)' '3' '51' '0.058823529411764705']
 ['Rotfrukter' '2' '21' '0.09523809523809523']
 ['Grönsaksblandningar med rotfrukter och eller baljväxter' '2' '9'
  '0.2222222222222222']
 ['Vatten mineralvatten' '1' '15' '0.06666666666666667']
 ['Sallad blandad mat' '1' '10' '0.1']
 ['Saft läsk cider u alkohol' '1' '19' '0.05263157894736842']
 ['Köttprodukter kötträtter' '1' '80' '0.0125']]
Kluster 1

[['Flingor - frukostflingor' '14' '28' '0.5']
 ['Hårt bröd' '8' '28' '0.2857142857142857']
 ['Godis ej choklad' '6' '11' '0.5454545454545454']
 ['Socker sirap honung' '4' '6' '0.6666666666666666']
 ['Bu

In [411]:
compliance = np.array(compliance)
compliance[5]

array([ 0.27941176,  0.64285714,  0.2       ,  0.33333333,  0.025     ,
        0.02898551,  1.        ,  1.        ,  0.0212766 ,  0.33333333])

## Läsa ner klustren i databasen

Kör inte nästa i onödan. Fyll på databasen kolumn för kolumn i stället

In [70]:
#Denna skapar tabellen kluster

conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()

curs.execute('drop table if exists kluster;')

# Backslash bara för radbrytningen, obs...
sql = 'CREATE TABLE IF NOT EXISTS kluster (Livsmedelsnummer TEXT PRIMARY KEY,\
kluster_3 INT DEFAULT -1, kluster_3_3 INT DEFAULT -1, kluster_20 INT DEFAULT -1,\
kluster_20_3 INT DEFAULT -1);'


curs.execute(sql)
conn.commit()

sql2 = 'INSERT INTO kluster (Livsmedelsnummer) SELECT Livsmedelsnummer from livs;'

curs.execute(sql2)
conn.commit()

conn.close()

In [79]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()

for index, Livsmedelsnummer in enumerate(dataset.T[1]):
    sql = 'UPDATE kluster SET kluster_3_3 = '+str(clusters[index])+' WHERE Livsmedelsnummer = "'+Livsmedelsnummer+'";'
    print(sql)
    curs.execute(sql)
#curs.execute('UPDATE livs SET Kluster = 1 WHERE Livsmedelsnummer = "2";')
conn.commit()
conn.close()

UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "1";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "2";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "3";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "4";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "5";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "6";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "7";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "8";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "9";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "10";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "12";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "13";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "17";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "18";
UPDATE kluster SET kluster_3_3 = 2 WHERE Livsmedelsnummer = "20";
UPDATE kluster SET 

UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1236";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1237";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1238";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1239";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1240";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1241";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1242";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1243";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1244";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1245";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1246";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1247";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1248";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmedelsnummer = "1249";
UPDATE kluster SET kluster_3_3 = 0 WHERE Livsmed

In [80]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
   
kluster_object = []
for row in curs.execute('SELECT * from kluster WHERE kluster_3_3 != -1;'):
    kluster_object.append(row)

        
kluster = np.array(kluster_object)

print(kluster)

conn.close()

[['1' '-1' '2' '-1' '-1']
 ['2' '-1' '2' '-1' '-1']
 ['3' '-1' '2' '-1' '-1']
 ..., 
 ['5964' '-1' '0' '-1' '-1']
 ['5973' '-1' '1' '-1' '-1']
 ['5974' '-1' '1' '-1' '-1']]


In [69]:
(dataset==str(kluster.T[0]))

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

## Visualisera med Plotly

In [ ]:
# transpose to work with Plotly-format (don't remember if it is sklearn.cluster or plotly that is counterintuitive)
dataToChart = dataset.T

# create multi dimensional array of data by label
segmentedData = [[[] for _ in range(3)] for _ in range(numClusters)]

for num, cluster in enumerate(clusters):
    if debug:
        print (str(num), str(clusters))
    segmentedData[cluster][0].append(dataToChart[0][num])
    segmentedData[cluster][1].append(dataToChart[1][num])
    segmentedData[cluster][2].append(dataToChart[2][num])

#if debug:
    #print(segmentedData)

In [69]:
#plotly.offline.init_notebook_mode(connected=True)
    
traces = []
baseColor = 100
i = 0
while i < numClusters:
    trace = go.Scatter3d(
        x=segmentedData[i][0],
        y=segmentedData[i][1],
        z=segmentedData[i][2],
        mode='markers',
        marker=dict(
            size=12,
            line=dict(
                color='rgba(baseColor+(i*2), baseColor+(i*2), baseColor+(i*2), 0.14)',
                width=0.5
            ),
            opacity=0.8
        ),
        # @todo: fix names list for plotly
        #text=names
    )
    traces.append(trace)
    i+=1

layout = go.Layout(
    scene=go.Scene(
        xaxis=go.XAxis(title='Carbs', tickprefix='Carbs ', showtickprefix='first'),
        yaxis=go.YAxis(title='Fat', tickprefix='Fat ', showtickprefix='first'),
        zaxis=go.ZAxis(title='Protein', tickprefix='Protein ', showtickprefix='first')
    ),
    height = 800,
    width = 800,
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='table1')